# Tutorial 2: Duffing Oscillator

This tutorial mirrors the 1D bistable walkthrough but applies it to the 2D Duffing oscillator.

The dynamics are
$$
\begin{aligned}
\dot{x} &= y, \\
\dot{y} &= -y + x - x^3.
\end{aligned}
$$

Key qualitative features:
- Two stable equilibria at $(\pm 1, 0)$
- One unstable equilibrium (the separatrix) at $(0, 0)$
- Damped oscillatory trajectories

## Learning Objectives
- Configure a 2D system using only its vector field
- Train Koopman eigenfunction models with the Separatrix Locator
- Visualise eigenfunctions via contour plots
- Compare a standard and "squashed" Koopman objective


In [ ]:
# Import necessary libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchdiffeq import odeint

# Import the separatrix locator package
import sys
sys.path.append('..')  # Add parent directory to path

from separatrix_locator import SeparatrixLocator
from separatrix_locator.distributions import MultivariateGaussian
from separatrix_locator.utils import plot_trajectories, plot_separatrix

from torch import nn

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 1. Define the Duffing Oscillator

We provide the vector field directly rather than relying on a `DynamicalSystem` helper.


In [ ]:
# Define the Duffing oscillator vector field
def dynamics_func(x: torch.Tensor) -> torch.Tensor:
    x = x.reshape(-1, 2)
    dxdt = torch.zeros_like(x)
    dxdt[:, 0] = x[:, 1]
    dxdt[:, 1] = -x[:, 1] + x[:, 0] - x[:, 0] ** 3
    return dxdt


dynamics_dim = 2
stable_points = torch.tensor([[1.0, 0.0], [-1.0, 0.0]])
separatrix_guess = torch.tensor([[0.0, 0.0]])

print(f"Dimension: {dynamics_dim}")
print(f"Stable equilibria: {stable_points.tolist()}")
print(f"Separatrix guess: {separatrix_guess.tolist()}")


In [ ]:
# Sample initial conditions and visualise the vector field
distribution = MultivariateGaussian(dim=dynamics_dim)
samples = distribution.sample(sample_shape=(2000,))

lin = torch.linspace(-2.0, 2.0, 25)
X_vals = lin.numpy()
Y_vals = lin.numpy()
X, Y = np.meshgrid(X_vals, Y_vals)  # X, Y shape (25, 25)

grid_points = np.stack([X.ravel(), Y.ravel()], axis=-1)  # shape (625, 2)
grid_points_torch = torch.tensor(grid_points, dtype=torch.float32)
field = dynamics_func(grid_points_torch)
U = field[:, 0].reshape(X.shape).numpy()
V = field[:, 1].reshape(Y.shape).numpy()

fig, axes = plt.subplots(1, 2, figsize=(9, 3.6))
# Use streamplot (requires rectilinear mesh)
axes[0].streamplot(X, Y, U, V, color='tab:blue', linewidth=1)
axes[0].set_aspect('equal')
axes[0].set_title('Vector field')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].scatter(stable_points[:, 0], stable_points[:, 1], c='tab:green', s=40, label='Stable')
axes[0].scatter(separatrix_guess[:, 0], separatrix_guess[:, 1], c='tab:red', s=60, label='Separatrix')
axes[0].legend()

axes[1].scatter(samples[:, 0].numpy(), samples[:, 1].numpy(), s=6, alpha=0.25)
axes[1].set_title('Initial condition distribution')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_aspect('equal')

plt.tight_layout()


### Train a Koopman Eigenfunction


In [ ]:
model = nn.Sequential(
    nn.Linear(dynamics_dim, 256),
    nn.Tanh(),
    nn.Linear(256, 256),
    nn.Tanh(),
    nn.Linear(256, 1),
)


In [ ]:
model.to(device)
locator = SeparatrixLocator(
    models=[model],
    dynamics_dim=dynamics_dim,
    device=device,
    verbose=True,
    epochs=4000,
)

locator.fit(
    func=dynamics_func,
    distribution=distribution,
    batch_size=2048,
    balance_loss_lambda=1e-2,
    eigenvalue=1.0,
    RHS_function="lambda phi: phi",
)


In [ ]:
# Visualize the learned eigenfunction on a grid

# Use torch.linspace to define a uniform grid over the state space
grid_lin = torch.linspace(-2.0, 2.0, 121)
X, Y = torch.meshgrid(grid_lin, grid_lin, indexing="ij")
coords = torch.stack((X, Y), dim=-1).reshape(-1, dynamics_dim).to(device)

with torch.no_grad():
    values = model(coords).cpu().numpy().reshape(X.shape)

X_np = X.cpu().numpy()
Y_np = Y.cpu().numpy()
fig, ax = plt.subplots(figsize=(5, 4))

# Plot filled contours for the Koopman eigenfunction
contourf = ax.contourf(X_np, Y_np, values, levels=30, cmap='coolwarm')
plt.colorbar(contourf, ax=ax, label='Koopman eigenfunction')

# Plot equilibria and separatrix guess
ax.scatter(stable_points[:, 0], stable_points[:, 1], c='tab:green', s=40, label='Stable equilibria')
ax.scatter(separatrix_guess[:, 0], separatrix_guess[:, 1], c='tab:red', s=60, label='Separatrix guess')

# Highlight zero contour (yellow, dashed, thick)
zero_contour = ax.contour(
    X_np, Y_np, values, levels=[0], colors=['yellow'], linewidths=3, linestyles='--'
)
if len(zero_contour.collections) > 0:
    zero_contour.collections[0].set_label('Zero contour')

# Overlay streamplot for the vector field using the same grid logic as in file_context_0
lin = torch.linspace(-2.0, 2.0, 25)
X_stream_vals = lin.numpy()
Y_stream_vals = lin.numpy()
X_stream, Y_stream = np.meshgrid(X_stream_vals, Y_stream_vals)
grid_points_stream = np.stack([X_stream.ravel(), Y_stream.ravel()], axis=-1)
grid_points_torch = torch.tensor(grid_points_stream, dtype=torch.float32).to(device)
field = dynamics_func(grid_points_torch)
U = field[:, 0].cpu().numpy().reshape(X_stream.shape)
V = field[:, 1].cpu().numpy().reshape(Y_stream.shape)

# Use streamplot with this rectilinear mesh
ax.streamplot(X_stream, Y_stream, U, V, color='tab:blue', linewidth=1)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Koopman eigenfunction level sets')
ax.set_aspect('equal')
# ax.legend()
plt.tight_layout()
plt.show()


### Squashed Koopman Eigenfunction


In [ ]:
model = nn.Sequential(
    nn.Linear(dynamics_dim, 256),
    nn.Tanh(),
    nn.Linear(256, 256),
    nn.Tanh(),
    nn.Linear(256, 1),
)

model.to(device)
locator = SeparatrixLocator(
    models=[model],
    dynamics_dim=dynamics_dim,
    device=device,
    verbose=True,
    epochs=2000,
)

locator.fit(
    func=dynamics_func,
    distribution=distribution,
    batch_size=2048,
    balance_loss_lambda=1e-2,
    eigenvalue=1.0,
    RHS_function="lambda phi: phi - phi**3",
)


In [ ]:
# Visualise the squashed eigenfunction

# Recreate a fine grid for the contour plot
axis_vals = np.linspace(-2.0, 2.0, 121, dtype=np.float32)
X_np, Y_np = np.meshgrid(axis_vals, axis_vals, indexing="ij")
coords_grid = torch.from_numpy(np.stack([X_np.ravel(), Y_np.ravel()], axis=1)).to(device)

with torch.no_grad():
    values = model(coords_grid).cpu().numpy().reshape(X_np.shape)

plt.figure(figsize=(5, 4))
contours = plt.contourf(X_np, Y_np, values, levels=30, cmap='coolwarm')
plt.colorbar(contours, label='Squashed eigenfunction')
plt.scatter(stable_points[:, 0], stable_points[:, 1], c='tab:green', s=40, label='Stable equilibria')
plt.scatter(separatrix_guess[:, 0], separatrix_guess[:, 1], c='tab:red', s=60, label='Separatrix guess')

# Brightly coloured zero contour
zero_contour = plt.contour(
    X_np, Y_np, values, levels=[0], colors=['yellow'], linewidths=3, linestyles='--'
)
if len(zero_contour.collections) > 0:
    zero_contour.collections[0].set_label('Zero contour')

# Overlay streamplot using the same method as in file_context_0
lin = torch.linspace(-2.0, 2.0, 25)
X_vals = lin.numpy()
Y_vals = lin.numpy()
X_stream, Y_stream = np.meshgrid(X_vals, Y_vals)  # X, Y shape (25, 25)

grid_points = np.stack([X_stream.ravel(), Y_stream.ravel()], axis=-1)  # shape (625, 2)
grid_points_torch = torch.tensor(grid_points, dtype=torch.float32, device=device)
with torch.no_grad():
    field = dynamics_func(grid_points_torch)
U = field[:, 0].reshape(X_stream.shape).cpu().numpy()
V = field[:, 1].reshape(Y_stream.shape).cpu().numpy()

plt.streamplot(X_stream, Y_stream, U, V, color='tab:blue', linewidth=0.7)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Squashed eigenfunction level sets')
plt.show()
